# Deploy the API Call and Timing Data Pipeline

## Extract Environment Settings

In [ ]:
import os

bucket = os.environ['CF_BUCKET']
region = os.environ['AWS_DEFAULT_REGION']
master_user_password = os.environ['MASTER_USER_PASSWORD']
key_name = os.environ['KEYPAIRNAME']


# Set bucket root based on region and deploy bucket
bucketRoot = 'https://s3-' + region + '.amazonaws.com/' + bucket
print bucketRoot

## Stack Names

In [ ]:
vpc_stack = 'calldata-vpc'
bucket_stack = 'calldata-bucket'
redshift_stack = 'calldata-redshift'
firehose_stack = 'calldata-firehose'
lambda_stack = 'calldata-lambda'

## Stack Settings

In [ ]:
database_name = 'mydb'
redshift_db_user = 'mydbuser'
db_bastion_host_stack = 'db-maint-host'
sumo_source_bucket = 'xt-sumo-test-eu-west-1'

## SDK Setup

In [ ]:
import boto3

client = boto3.client('cloudformation')
waiter = client.get_waiter('stack_create_complete')

## VPC Set Up

In [ ]:
response = client.create_stack(
    StackName=vpc_stack,
    TemplateURL='https://s3-' + region + '.amazonaws.com/' + bucket + '/vpc-pub-priv.yaml'
)

print response

print 'waiting for network stack to complete...'
waiter.wait(
    StackName=vpc_stack
)
print 'network stack created'

## Harvest VPC Values

In [ ]:
response = client.describe_stacks(
    StackName=vpc_stack
)

stack_outputs = response['Stacks'][0]['Outputs']

vpc = [d for d in stack_outputs if d['OutputKey'] == 'VpcId'][0]['OutputValue']
print vpc
privateSubnet1 = [d for d in stack_outputs if d['OutputKey'] == 'PrivateSubnet1'][0]['OutputValue']
print privateSubnet1
privateSubnet2 = [d for d in stack_outputs if d['OutputKey'] == 'PrivateSubnet2'][0]['OutputValue']
print privateSubnet2
publicSubnet1 = [d for d in stack_outputs if d['OutputKey'] == 'PublicSubnet1'][0]['OutputValue']
print publicSubnet1
publicSubnet2 = [d for d in stack_outputs if d['OutputKey'] == 'PublicSubnet2'][0]['OutputValue']
print publicSubnet2

## Create Redshift Cluster

In [ ]:
response = client.create_stack(
    StackName=redshift_stack,
    TemplateURL='https://s3-' + region + '.amazonaws.com/' + bucket + '/redshift.yml',
    Parameters=[
        {
            'ParameterKey': 'MasterUsername',
            'ParameterValue': redshift_db_user
        },
        {
            'ParameterKey': 'DatabaseName',
            'ParameterValue': database_name
        },
        {
            'ParameterKey': 'NodeType',
            'ParameterValue': 'dw2.large'
        },
        {
            'ParameterKey': 'VpcId',
            'ParameterValue': vpc
        },
        {
            'ParameterKey': 'PublicSubnet1',
            'ParameterValue': publicSubnet1
        },
        {
            'ParameterKey': 'PublicSubnet2',
            'ParameterValue': publicSubnet2
        },
        {
            'ParameterKey': 'MasterUserPassword',
            'ParameterValue': master_user_password
        }
    ]
)

print response

In [ ]:
print 'waiting for redshift stack to complete...'
waiter.wait(
    StackName=redshift_stack
)
print 'redshift stack created'

## Extract Redshift Stack Output

In [ ]:
response = client.describe_stacks(
    StackName=redshift_stack
)

stack_outputs = response['Stacks'][0]['Outputs']

redshift_vpc_sg = [d for d in stack_outputs if d['OutputKey'] == 'VPCIngressSecurityGroup'][0]['OutputValue']
print redshift_vpc_sg

redshift_cluster_hostname = [d for d in stack_outputs if d['OutputKey'] == 'ClusterHostName'][0]['OutputValue']
print redshift_cluster_hostname

## Launch Bastion Host

Launch a bastion host into the same VPC as the redshift DB, and leverage user data to create the tables needed in Redshift.

In [ ]:
templateUrl = bucketRoot + '/launch_maint.yml'
print 'Launch bastion host via', templateUrl

launch_resp = client.create_stack(
    StackName=db_bastion_host_stack,
    TemplateURL=templateUrl,
    Parameters=[
        {
            'ParameterKey': 'PublicSubnet',
            'ParameterValue': publicSubnet1
        },
        {
            'ParameterKey': 'DBMaintSecurityGroup',
            'ParameterValue': redshift_vpc_sg
        }
        ,
        {
            'ParameterKey': 'KeyName',
            'ParameterValue': key_name
        },
        {
            'ParameterKey': 'DBHost',
            'ParameterValue': redshift_cluster_hostname
        },
        {
            'ParameterKey': 'DBUser',
            'ParameterValue': redshift_db_user
        },
        {
            'ParameterKey': 'DBPassword',
            'ParameterValue': master_user_password
        },
        {
            'ParameterKey': 'DBService',
            'ParameterValue': database_name
        },
        {
            'ParameterKey': 'BucketName',
            'ParameterValue': bucket
        }
    ],
    Capabilities=[
        'CAPABILITY_IAM',
    ]
)

print launch_resp

## Firehose Bucket

Firehose needs an S3 bucket to stage the data it loads into redshift. Here we create it with cloud formation so we can set up and tear down buckets based on the stack, so we don't worry about bucket name, etc.

In [ ]:
response = client.create_stack(
    StackName=bucket_stack,
    TemplateURL=bucketRoot +  '/bucket.yml',
)

print response

In [ ]:
print 'waiting for bucket stack to complete...'
waiter.wait(
    StackName=bucket_stack
)
print 'bucket created'

In [ ]:
response = client.describe_stacks(
    StackName=bucket_stack
)

stack_outputs = response['Stacks'][0]['Outputs']

firehose_bucket_name = [d for d in stack_outputs if d['OutputKey'] == 'BucketName'][0]['OutputValue']
print firehose_bucket_name

## Firehose Stack

In [ ]:
jdbc_url = 'jdbc:redshift://' + redshift_cluster_hostname + ':5439/' + database_name
print jdbc_url

In [ ]:
response = client.create_stack(
    StackName=firehose_stack,
    TemplateURL=bucketRoot +  '/firehose.yml',
    Parameters=[
        {
            'ParameterKey': 'FirehoseBucketName',
            'ParameterValue': firehose_bucket_name
        },
        {
            'ParameterKey': 'RedshiftJDBCUrl',
            'ParameterValue': jdbc_url
        },
        {
            'ParameterKey': 'Username',
            'ParameterValue': redshift_db_user
        },
        {
            'ParameterKey': 'Password',
            'ParameterValue':master_user_password
        }
    ],
    Capabilities=[
        'CAPABILITY_IAM',
    ]
)

print response

In [ ]:
print 'waiting for firehose stack to complete...'
waiter.wait(
    StackName=firehose_stack
)
print 'firehose created'

In [ ]:
response = client.describe_stacks(
    StackName=firehose_stack
)

stack_outputs = response['Stacks'][0]['Outputs']

svc_call_stream = [d for d in stack_outputs if d['OutputKey'] == 'SvcCallDeliveryStream'][0]['OutputValue']
print svc_call_stream
call_record_stream = [d for d in stack_outputs if d['OutputKey'] == 'CallRecordDeliveryStream'][0]['OutputValue']
print call_record_stream

## Put a Record

In [ ]:
firehose_client = boto3.client('firehose')

response = firehose_client.put_record(
    DeliveryStreamName=call_record_stream,
    Record={
        'Data': b'2016-07-01 15:22:58|181c575a-ef8c-4468-76e3-3c95ff3a5e4b|false|vc2coma2078845n|/xapi/DEV/NONPROD|xtracApi-GET-work-items-communications|XWHRon|a79fcb28-2621-4973-8a1e-c09a2ab30f79|262\n'
    }
)

print response

In [ ]:
response = firehose_client.put_record(
    DeliveryStreamName=svc_call_stream,
    Record={
        'Data':b'2016-09-09 15:09:09|181c575a-ef8c-4468-76e3-3c95ff3a5e4b|false|Core-Correspondence-RetrieveCorrList|vc2coma2078845n.fmr.com:11000|15\n'
    }
)

print response

## Deploy Lambda

In [ ]:
response = client.create_stack(
    StackName=lambda_stack,
    TemplateURL=bucketRoot +  '/sumoreader.yml',
    Parameters=[
        {
            'ParameterKey': 'CallRecordStreamName',
            'ParameterValue': call_record_stream
        },
        {
            'ParameterKey': 'SvcCallStreamName',
            'ParameterValue': svc_call_stream
        },
        {
            'ParameterKey': 'CodeBucketName',
            'ParameterValue': bucket
        },
        {
            'ParameterKey': 'CodeFileName',
            'ParameterValue': 'lambda.zip'
        },
        {
            'ParameterKey': 'SumoSourceBucketName',
            'ParameterValue': sumo_source_bucket
        }
        
    ],
    Capabilities=[
        'CAPABILITY_IAM',
    ]
)

print response

In [ ]:
foo = """
{
    "foo":123
}"""
    
print foo

In [ ]:
response = client.describe_stacks(
    StackName=lambda_stack
)

stack_outputs = response['Stacks'][0]['Outputs']

lambda_arn = [d for d in stack_outputs if d['OutputKey'] == 'SumoReaderLambdaArn'][0]['OutputValue']
print lambda_arn

In [ ]:
event_config_open = """
{
    "LambdaFunctionConfigurations": [{
        "Id":"invoke_parse_fn",
        "LambdaFunctionArn": \""""

event_config_close = """\",
        "Events": [
            "s3:ObjectCreated:*"
        ]
    }]
}
"""    
    
event_config = event_config_open + 'lambda_arn' + event_config_close

print event_config

In [ ]:
s3client = boto3.client('s3')
response = s3client.put_bucket_notification_configuration(
    Bucket=sumo_source_bucket,
    NotificationConfiguration={
        'LambdaFunctionConfigurations': [
            {
                'LambdaFunctionArn': lambda_arn,
                'Events': [
                    's3:ObjectCreated:*'
                ],
            }
        ]
    })

print response